In [2]:
#! /usr/bin/python3

# Script for comparing an input .csv file with an existing .csv file (e.g. the current CHGIS).
# Indicates 1) matches on name and 2) strength of match on content

import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import os.path

In [4]:
def name_checker(name):
    '''
       Confirms that pathname is a valid file 
    '''
    while not (os.path.isfile(name)):
        print("Not a valid filename.  Please try again.")
        name = input()
    return name

In [5]:
def csv_picker():
    '''
        Simple function that takes the filenames / paths of the new .csv to check and the old .csv (i.e. a CHGIS version) against which to check it.
        Returns a list from which the DataFrames can then be constructed.
    '''
    print("Please type in the path of the file (with extension) you wish to check against the CHGIS:")
    new_csv = input()
    new_csv = name_checker(new_csv)
    #while !os.path.isfile(new_csv):
    #    print("Not a valid filename.  Please try again:")
    #    new_csv = input()
    print("Please type in the path of the file (with extension) containing the version of the CHGIS you wish to check against:")
    old_csv = input()
    old_csv = name_checker(old_csv)
    return [new_csv, old_csv]


In [7]:
# def field_checker(new_csv, old_csv):
#    '''
#        Contains the logic for checking fields of the new_csv against those of the old_csv 
#        Presumes the existence of the following cols:       
#    '''
    

In [15]:
### start of test to see if Donghan_2014-10-02_copy.csv as new, chgis v5 as old works

# importing .csv files
dong_han = pd.read_csv('../input/sample_data/Donghan_2014-10-02_copy.csv', low_memory=False)
v5 = pd.read_csv('../input/v5_augment_2016-08-09.csv', low_memory=False)

# tagging source files
dong_han['data_source'] = 'dong_han'
v5['data_source'] = 'v5'

# appending 县 to '县名' column, to prepare data for import into v5
dong_han['县名'] = dong_han['县名'] + '县'

# renaming cols of dong_han to match v5
dong_han = dong_han.rename(columns={
        '县名':'nm_simp',
        '經度':'x_coord',
        '緯度':'y_coord',
        '屬':'prnt_simp',
        'BEG':'beg',
        'END':'end'        
    })

# concatenating into a single DF
df = pd.concat([dong_han, v5])
#df.to_csv('../output/160829_test.csv')

In [16]:
# checking for duplicated names
mask_duplicates = df.duplicated(subset='nm_simp', keep=False)
mask_uniques = ~mask_duplicates
duplicates = df[mask_duplicates]
duplicates['nm_simp_match'] = True
uniques = df[mask_uniques]
uniques['nm_simp_match'] = False
df_two = pd.concat([duplicates, uniques])

/home/sf/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/sf/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [21]:
# function for comparing content fields
def field_matcher(frame, comp_field, indicator_field):
    mask_duplicates = frame.duplicated(subset=[comp_field], keep=False)
    mask_uniques = ~mask_duplicates
    duplicates = frame[mask_duplicates]
    duplicates[indicator_field] = True
    uniques = frame[mask_uniques]
    uniques[indicator_field] = False
    return pd.concat([duplicates, uniques])

In [22]:
# checking content matches

# trying to workaround failure to recognize numerical matches
df_two['x_coord'] = df_two['x_coord'].astype(str)
df_two['y_coord'] = df_two['y_coord'].astype(str)
df_two['beg'] = df_two['beg'].astype(str)
df_two['end'] = df_two['end'].astype(str)

# creating flag fields for content matches
df_three = field_matcher(df_two, 'x_coord', 'x_coord_match')
df_four = field_matcher(df_three,'y_coord', 'y_coord_match')
df_five = field_matcher(df_four,'beg', 'beg_match')
df_six = field_matcher(df_five,'end', 'end_match')

/home/sf/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/sf/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [29]:
### RESUME HERE IN AM -- https://stackoverflow.com/questions/25748683/python-pandas-sum-dataframe-rows-for-given-columns

content_matches = [df_six['x_coord_match'], df_six['y_coord_match'], df_six['beg_match'], df_six['end_match']]
df_six['match_strength'] = df[content_matches].sum(axis=0)

ValueError: Item wrong length 4 instead of 78850.